# Загрузка библиотек

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers[sentencepiece]
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.8 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00


In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00


# Импорты

In [ ]:
import optuna
import pickle
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import VotingClassifier

import transformers
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

from datasets import load_metric, Dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка данных

Так как задача ресурсоёмкая, возьмем датасет меньшего размера.

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/automl/train_balanced.csv')
data.head()

,Unnamed: 0,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,23304,aafad5bf4f34,761B16CEE6C1,its unfair,Claim,Adequate
1,12749,ae9adef651a2,E60A91172B20,Luke should use these reason's to convince oth...,Position,Adequate
2,8378,d0daccd97f67,974383195606,During your time in musical you have to memori...,Evidence,Adequate
3,32112,798777fe3fa6,F309843EDF56,2000 fiasco which was the biggest election cri...,Evidence,Adequate
4,12819,8e87b718694d,E71135F11EE0,people are just down right lazy and they need ...,Claim,Adequate


In [ ]:
del data['Unnamed: 0']

In [ ]:
label_mapping = {'Effective': 2, 'Adequate': 1, 'Ineffective': 0}
data['discourse_effectiveness_map'] = data['discourse_effectiveness'].map(label_mapping)

In [ ]:
data

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,discourse_effectiveness_map
0,aafad5bf4f34,761B16CEE6C1,its unfair,Claim,Adequate,1
1,ae9adef651a2,E60A91172B20,Luke should use these reason's to convince oth...,Position,Adequate,1
2,d0daccd97f67,974383195606,During your time in musical you have to memori...,Evidence,Adequate,1
3,798777fe3fa6,F309843EDF56,2000 fiasco which was the biggest election cri...,Evidence,Adequate,1
4,8e87b718694d,E71135F11EE0,people are just down right lazy and they need ...,Claim,Adequate,1
...,...,...,...,...,...,...
8995,886e6203d687,19AEDCFA6EF1,This means that most candidates do not spend t...,Evidence,Ineffective,0
8996,91435c958d00,51CD300AB688,Think about it would any of this happen if the...,Claim,Ineffective,0
8997,fe38c88f6925,20D57DEC3EF5,"In conclusion, I think classrooms shouldn't us...",Concluding Statement,Ineffective,0
8998,bc6b5d48536e,1A26AFD79998,The mona lisa demonstration is one of the pain...,Claim,Ineffective,0


In [ ]:
train, test = train_test_split(
    data,
    test_size=0.3,
    stratify = data.discourse_effectiveness_map,
    random_state=42
)

In [ ]:
train.shape

(6300, 6)

In [ ]:
test.shape

(2700, 6)

# Transformers

BERT в LAMA уже показал отличный результат. Что можно придумать лучше, чем оди трансформер? Взять два трансформера и собрать их в ансамбль. Для этого возьмем 2 модели: distilbert (по данным с kaggle он давал лучшие результаты) и roberta.

In [ ]:
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

class HuggingFaceClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, model_name='bert-base-uncased', num_labels=3):
        self.model_name = model_name
        self.num_labels = num_labels
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def objective(self, trial, model, tokenizer, train_dataset, val_dataset):
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)

        training_args = TrainingArguments(
            output_dir='/content/drive/MyDrive/automl/opt',
            learning_rate=learning_rate,
            num_train_epochs=3,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=32,
            evaluation_strategy="epoch",
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics
        )

        trainer.train()

        eval_result = trainer.evaluate()
        metric_to_optimize = eval_result["eval_accuracy"]

        return metric_to_optimize

    def fit(self, X, y):
        train_data_new = X[X['is_test'] == False]
        val_data_new = X[X['is_test']]

        train_data_new.loc[:, ('label')] = y[:len(train_data_new)]
        train_inputs = self.tokenizer(train_data_new['discourse_text'].tolist(), padding=True, truncation=True, return_tensors="pt")
        train_data_new['input_ids'] = pd.Series(train_inputs['input_ids'].tolist())
        train_data_new['attention_mask'] = pd.Series(train_inputs['attention_mask'].tolist())
        tds = Dataset.from_pandas(train_data_new)

        val_data_new.loc[:, ('label')] = y[len(train_data_new):]
        val_inputs = self.tokenizer(val_data_new['discourse_text'].tolist(), padding=True, truncation=True, return_tensors="pt")
        val_data_new['input_ids'] = pd.Series(val_inputs['input_ids'].tolist())
        val_data_new['attention_mask'] = pd.Series(val_inputs['attention_mask'].tolist())
        vds = Dataset.from_pandas(train_data_new)

        model = AutoModelForSequenceClassification.from_pretrained(self.model_name, num_labels=self.num_labels)


        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: self.objective(trial, model, self.tokenizer, Dataset.from_pandas(train_data_new.sample(frac=1)), vds), n_trials=2)
        best_params = study.best_params
        print(f"Best parameters for {self.model_name}: {best_params}")
        print(f"Training {self.model_name}...")

        training_args = TrainingArguments(
            output_dir='/content/drive/MyDrive/automl/results',
            learning_rate=best_params['learning_rate'],
            num_train_epochs=3,
            per_device_train_batch_size=8,
            warmup_steps=500,
            weight_decay=0.01,
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tds,
            eval_dataset=vds,
        )

        trainer.train()

        self.model = model

        return self

    def predict(self, data):
        inputs = self.tokenizer(data['discourse_text'].tolist(), padding=True, truncation=True, return_tensors="pt")

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = self.model.to(device)
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()

        return predictions

    def predict_proba(self, data):
        inputs = self.tokenizer(data['discourse_text'].tolist(), padding=True, truncation=True, return_tensors="pt")

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = self.model.to(device)
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        probabilities = F.softmax(outputs.logits, dim=-1)

        probabilities = probabilities.cpu().numpy()

        return probabilities

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
pd.options.mode.chained_assignment = None

classifiers = [
    ('model1', HuggingFaceClassifier(model_name='distilbert-base-uncased')),
    ('model2', HuggingFaceClassifier(model_name='roberta-base'))
]

ensemble = VotingClassifier(estimators=classifiers, voting='soft')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
train['is_test'] = False
test['is_test'] = True

data = pd.concat([train, test], ignore_index=True)

ensemble.fit(data[['discourse_text', 'is_test']], data['discourse_effectiveness_map'])

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2023-12-28 11:07:19,364] A new study created in memory with name: no-name-4477e5ce-bcc1-469f-97be-393b7fca6b22
<ipython-input-42-10d9f5b8bcd9>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.920600,0.726410,0.670000
2,0.663900,0.384341,0.862698
3,0.422600,0.204076,0.931429


[I 2023-12-28 11:30:34,009] Trial 0 finished with value: 0.9314285714285714 and parameters: {'learning_rate': 5.104948262531621e-05}. Best is trial 0 with value: 0.9314285714285714.
<ipython-input-42-10d9f5b8bcd9>:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-4)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.230900,0.107115,0.970000
2,0.100000,0.069507,0.981905
3,0.073600,0.056137,0.987143


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.516900,0.435926,0.829841
2,0.370500,0.298698,0.915397
3,0.262200,0.340396,0.910476


[I 2023-12-28 13:38:56,321] Trial 1 finished with value: 0.9104761904761904 and parameters: {'learning_rate': 1.9829007717910955e-05}. Best is trial 1 with value: 0.9104761904761904.


Best parameters for roberta-base: {'learning_rate': 1.9829007717910955e-05}
Training roberta-base...


Step,Training Loss
500,0.385200
1000,0.419300
1500,0.379600
2000,0.251200


VotingClassifier(estimators=[('model1',
                              HuggingFaceClassifier(model_name='distilbert-base-uncased')),
                             ('model2',
                              HuggingFaceClassifier(model_name='roberta-base'))],
                 voting='soft')

In [ ]:
pickle.dump(ensemble, open('/content/drive/MyDrive/automl/ensemble_model.pkl', 'wb'))

In [ ]:
ensemble = pickle.load(open('/content/drive/MyDrive/automl/ensemble_model.pkl', 'rb'))

In [ ]:
predictions = ensemble.predict_proba(test)

In [56]:
print('TEST score: {}'.format(log_loss(test['discourse_effectiveness_map'].values, predictions))

TEST score: 0.9746582647658
